# 要約 
このJupyterノートブックでは、Gemma-2 9bモデルのトレーニング方法とその結果について説明されています。このノートブックの主な目的は、大規模言語モデル（LLM）のファインチューニングにおける最新の手法であるQLoRA (Quantized Low-Rank Adaptation)を使用して、競技会でのパフォーマンスを向上させることです。

### 問題点
ノートブックは、ユーザーが好みのチャットボット応答を予測するために、Gemma-2のトレーニングによる評価パフォーマンスを改善しようとしています。具体的には、評価セットのロスを最小化することで、実際の競技会や評価基準のスコア（LB: 0.941）を向上させることを目指しています。

### 使用手法とライブラリ
以下の手法およびライブラリが使用されています。

1. **QLoRA**: 低ランクの適応法を用いてモデルのトレーニングを効率化し、大規模モデルのメモリ使用を減らします。モデルの重みを量子化しつつトレーニングを行うため、計算コストを削減します。
2. **LoRA**: 通常のファインチューニングの代わりに、特定の層に小さな行列を介してパラメータを調整することで、トレーニング中に元の重みの更新を最小限に抑えます。
3. **Hugging Face Transformers**: 一般的なトランスフォーマーモデルのフレームワークとして使用され、`Gemma2ForSequenceClassification`などの特定のモデルやトークナイザーが利用されています。
4. **PyTorch**: モデルのトレーニングと評価に使用される深層学習ライブラリです。
5. **Sklearn**: 精度や対数損失の計算を行うために用いられています。

### ノートブックのフロー
- **データ準備**: Kaggleのデータセットを読み込み、カスタムトークナイザーを設定してデータを前処理します。
- **モデル設定**: Gemma-2の設定やトレーニングメソッドを指定し、LoRAおよびQLoRAのパラメータを設定します。
- **トレーニング**: 計算されたいくつかの評価指標（例えば、ロスと精度）を用いて、モデルのトレーニングを実施します。
- **結果の評価**: 最後に、評価セットに対するモデルのロスと精度を報告します。

このノートブックは、全体のプロセスを通じて、LLMのファインチューニングを改善し、実用的なパフォーマンスを提供するための具体的なステップと結果を示しています。

---


# 用語概説 
以下は、ノートブックに含まれる専門用語に関する簡単な解説です。初心者がつまずきそうな、実務経験が少ないと馴染みのない用語に焦点を当てています。

1. **LoRA (Low-Rank Adaptation)**:
   LoRAは、モデルの重みを効率的に更新する技術で、大規模なモデルに対してファインチューニングを行う際、重みの更新を行うのではなく、低ランクの行列を利用して重みの変化を表現します。これにより、メモリ使用量が削減され、トレーニングの効率が向上します。

2. **QLoRA (Quantized Low-Rank Adaptation)**:
   LoRAの拡張で、量子化技術を用いてモデルの重みをさらに低精度で圧縮します。これにより、大規模モデルの必要なメモリ量を大幅に減少させながら、通常のファインチューニングに匹敵する性能を保持します。

3. **モデルの凍結 (Freezing a model)**:
   特定のパラメータや層をトレーニング中に固定し、学習を行わないようにすることです。これにより、トレーニング時のメモリ使用量を削減し、新しく追加されたパラメータにのみ学習を集中させることができます。

4. **データコレーター (Data Collator)**:
   バッチ処理の際にデータを整形する役割を持つ機能です。複数の入力をまとめて一つのバッチとして扱うために必要な前処理を行います。特にパディング（長さを揃える作業）などを自動で行ないます。

5. **バッチサイズ (Batch Size)**:
   モデルが一度に処理するデータの数を指します。大きいバッチサイズを設定することで、トレーニングのスピードを向上させることができますが、メモリ負荷が増す可能性もあります。一方、小さいバッチサイズはメモリの負担を軽減しますが、トレーニングが遅くなることがあります。

6. **勾配蓄積 (Gradient Accumulation)**:
   小さいバッチサイズで複数回のパスを行い、その勾配を累積して、最後に一度にパラメータを更新する手法です。これにより、実質的に大きなバッチサイズを使用したかのような効果を得ることができます。

7. **ドロップアウト (Dropout)**:
   ニューラルネットワークのトレーニング時に一部のニューロンをランダムに無効化する手法です。これは過学習を防ぐために有効で、モデルの一般化性能を向上させる効果があります。

8. **トークナイザー (Tokenizer)**:
   テキストデータをモデルが処理しやすい形式に変換するためのツールです。テキストを単語やサブワードに分割し、数値として表現します。

9. **フォワード/バックワード計算**:
   フォワード計算は、モデルに入力を与えた時の出力を計算するプロセスです。バックワード計算は誤差逆伝播法を利用して、出力の誤りに基づいてモデルの重みを更新するプロセスです。

10. **ファインチューニング (Fine-tuning)**:
   すでにトレーニングされたモデルを特定のタスクに合わせて追加のトレーニングを行うプロセスです。これにより、モデルは新しいデータに対してより適した性能を発揮できます。

これらの用語は、初学者が理解を深める上で重要なものですので、しっかりと理解しておくと良いでしょう。

---


## このノートブックについて
このノートブックでは、Gemma-2 9bをどのようにトレーニングしてLB: 0.941を取得したかを示します。推論コードは[こちら](https://www.kaggle.com/code/emiz6413/inference-gemma-2-9b-4-bit-qlora)にあります。
私は、unslothチームがアップロードした4ビット量子化された[Gemma 2 9b Instruct](https://huggingface.co/unsloth/gemma-2-9b-it-bnb-4bit)をベースモデルとして使用し、LoRAアダプターを追加して1エポックのトレーニングを行いました。

## 結果

評価セットとして`id % 5 == 0`を使用し、残りをすべてトレーニングに使用しました。

| サブセット | ロス |
| - | - |
| eval | 0.9371 |
| LB | 0.941 |

## QLoRAファインチューニングとは？

従来のファインチューニングでは、重み（$\mathbf{W}$）は次のように更新されます：

$$
\mathbf{W} \leftarrow \mathbf{W} - \eta \frac{{\partial L}}{{\partial \mathbf{W}}} = \mathbf{W} + \Delta \mathbf{W}
$$

ここで、$L$はこのステップでの損失、$\eta$は学習率です。

[LoRA](https://arxiv.org/abs/2106.09685)は、$\Delta \mathbf{W} \in \mathbb{R}^{\text{d} \times \text{k}}$を2つの（はるかに）小さな行列、$\mathbf{B} \in \mathbb{R}^{\text{d} \times \text{r}}$と$\mathbf{A} \in \mathbb{R}^{\text{r} \times \text{k}}$に因子分解して近似しようとします。ここで$r \ll \text{min}(\text{d}, \text{k})$です。

$$
\Delta \mathbf{W}_{s} \approx \mathbf{B} \mathbf{A}
$$

<img src="https://storage.googleapis.com/pii_data_detection/lora_diagram.png">

トレーニング中、元の重みを凍結し、$\mathbf{A}$と$\mathbf{B}$のみが更新されるため、トレーニング中に更新する必要がある元の重みの割合はごくわずか（例：<1%）です。この方式により、トレーニング中のGPUメモリ使用量を大幅に削減しながら、通常の（フル）ファインチューニングと同じ性能を達成できます。

[QLoRA](https://arxiv.org/abs/2305.14314)は、LLMを量子化することでさらに効率を高めています。例えば、8Bパラメータモデルは32ビットで32GBのVRAMを占有しますが、量子化された8ビット/4ビットの8Bモデルはそれぞれ8GB/4GBしか必要ありません。
QLoRAは、LLMの重みを低精度（例：8ビット）で量子化する一方で、フォワード/バックワードの計算は高精度（例：16ビット）で行い、LoRAアダプターの重みも高精度で維持されることに注意してください。

4ビットでのA6000使用の1エポックは約15時間かかり、8ビットでは約24時間かかりましたが、ロスの違いは目立ちませんでした。

## 注意
Kaggleカーネルでの完全なトレーニングの実行には非常に長い時間がかかります。完全なトレーニングを実行するためには外部の計算リソースを使用することをお勧めします。
このノートブックではデモ目的でわずか100サンプルを使用していますが、その他はすべて私の設定と同じです。


In [ ]:
# gemma-2はtransformers>=4.42.3から使用可能です
!pip install -U "transformers>=4.42.3" bitsandbytes accelerate peft

In [ ]:
import os
import copy
from dataclasses import dataclass

import numpy as np
import torch
from datasets import Dataset
from transformers import (
    BitsAndBytesConfig,
    Gemma2ForSequenceClassification,
    GemmaTokenizerFast,
    Gemma2Config,
    PreTrainedTokenizerBase, 
    EvalPrediction,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
from sklearn.metrics import log_loss, accuracy_score

### 設定


In [ ]:
@dataclass
class Config:
    output_dir: str = "output"  # 出力フォルダ
    checkpoint: str = "unsloth/gemma-2-9b-it-bnb-4bit"  # 4ビット量子化されたgemma-2-9b-instruct
    max_length: int = 1024  # 最大長
    n_splits: int = 5  # データ分割数
    fold_idx: int = 0  # 現在のフォールドインデックス
    optim_type: str = "adamw_8bit"  # 使用するオプティマイザの種類
    per_device_train_batch_size: int = 2  # デバイスごとのトレーニングバッチサイズ
    gradient_accumulation_steps: int = 2  # グローバルバッチサイズは8
    per_device_eval_batch_size: int = 8  # デバイスごとの評価バッチサイズ
    n_epochs: int = 1  # エポック数
    freeze_layers: int = 16  # 総レイヤ数は42、最初の16レイヤにはアダプターを追加しない
    lr: float = 2e-4  # 学習率
    warmup_steps: int = 20  # ウォームアップステップ数
    lora_r: int = 16  # LoRAのランク
    lora_alpha: float = lora_r * 2  # LoRAのアルファ
    lora_dropout: float = 0.05  # LoRAのドロップアウト率
    lora_bias: str = "none"  # LoRAのバイアス

config = Config()

#### トレーニング引数


In [ ]:
training_args = TrainingArguments(
    output_dir="output",  # 出力ディレクトリ
    overwrite_output_dir=True,  # 出力ディレクトリを上書きする
    report_to="none",  # ログの報告先
    num_train_epochs=config.n_epochs,  # トレーニングエポック数
    per_device_train_batch_size=config.per_device_train_batch_size,  # デバイスごとのトレーニングバッチサイズ
    gradient_accumulation_steps=config.gradient_accumulation_steps,  # グラデーション蓄積ステップ数
    per_device_eval_batch_size=config.per_device_eval_batch_size,  # デバイスごとの評価バッチサイズ
    logging_steps=10,  # ログの出力間隔
    eval_strategy="epoch",  # 評価戦略
    save_strategy="steps",  # 保存戦略
    save_steps=200,  # 保存ステップ数
    optim=config.optim_type,  # オプティマイザの種類
    fp16=True,  # FP16を使用するか
    learning_rate=config.lr,  # 学習率
    warmup_steps=config.warmup_steps,  # ウォームアップステップ数
)

#### LoRA設定


In [ ]:
lora_config = LoraConfig(
    r=config.lora_r,  # LoRAのランク
    lora_alpha=config.lora_alpha,  # LoRAのアルファ
    # 自己注意のみにターゲットを絞る
    target_modules=["q_proj", "k_proj", "v_proj"],  # ターゲットモジュール
    layers_to_transform=[i for i in range(42) if i >= config.freeze_layers],  # 変換対象レイヤー
    lora_dropout=config.lora_dropout,  # LoRAのドロップアウト率
    bias=config.lora_bias,  # LoRAのバイアス
    task_type=TaskType.SEQ_CLS,  # タスクの種類
)

### トークナイザーとモデルのインスタンス化


In [ ]:
tokenizer = GemmaTokenizerFast.from_pretrained(config.checkpoint)  # トークナイザーの読み込み
tokenizer.add_eos_token = True  # <eos>を末尾に追加
tokenizer.padding_side = "right"  # パディングを右側に設定

In [ ]:
model = Gemma2ForSequenceClassification.from_pretrained(
    config.checkpoint,  # モデルの読み込み
    num_labels=3,  # 分類ラベルの数
    torch_dtype=torch.float16,  # データ型の設定
    device_map="auto",  # デバイスマッピング
)
model.config.use_cache = False  # キャッシュを使用しない
model = prepare_model_for_kbit_training(model)  # kビットトレーニングのためのモデル準備
model = get_peft_model(model, lora_config)  # LoRAモデルの取得
model  # モデルを表示

In [ ]:
model.print_trainable_parameters()  # トレーニング可能なパラメータを表示

### データセットのインスタンス化


In [ ]:
ds = Dataset.from_csv("/kaggle/input/lmsys-chatbot-arena/train.csv")  # CSVからデータセットを作成
ds = ds.select(torch.arange(100))  # デモ目的で最初の100データのみを使用

In [ ]:
class CustomTokenizer:
    def __init__(
        self, 
        tokenizer: PreTrainedTokenizerBase, 
        max_length: int
    ) -> None:
        self.tokenizer = tokenizer  # トークナイザーの初期化
        self.max_length = max_length  # 最大長の設定
        
    def __call__(self, batch: dict) -> dict:
        prompt = ["<prompt>: " + self.process_text(t) for t in batch["prompt"]]  # プロンプトの整形
        response_a = ["\n\n<response_a>: " + self.process_text(t) for t in batch["response_a"]]  # 応答Aの整形
        response_b = ["\n\n<response_b>: " + self.process_text(t) for t in batch["response_b"]]  # 応答Bの整形
        texts = [p + r_a + r_b for p, r_a, r_b in zip(prompt, response_a, response_b)]  # プロンプトと応答を結合
        tokenized = self.tokenizer(texts, max_length=self.max_length, truncation=True)  # トークナイズ
        labels=[]
        for a_win, b_win in zip(batch["winner_model_a"], batch["winner_model_b"]):
            if a_win:  # モデルAが勝った場合
                label = 0  # ラベルは0
            elif b_win:  # モデルBが勝った場合
                label = 1  # ラベルは1
            else:  # 同点の場合
                label = 2  # ラベルは2
            labels.append(label)  # ラベルをリストに追加
        return {**tokenized, "labels": labels}  # トークン化されたデータとラベルを返す
        
    @staticmethod
    def process_text(text: str) -> str:
        return " ".join(eval(text, {"null": ""}))  # テキストを処理して返す

In [ ]:
encode = CustomTokenizer(tokenizer, max_length=config.max_length)  # カスタムトークナイザーのインスタンス化
ds = ds.map(encode, batched=True)  # データセットにマッピング

### 指標の計算

LBで使用するロスと補助的な指標として精度を計算します。


In [ ]:
def compute_metrics(eval_preds: EvalPrediction) -> dict:
    preds = eval_preds.predictions  # 予測値の取得
    labels = eval_preds.label_ids  # ラベルの取得
    probs = torch.from_numpy(preds).float().softmax(-1).numpy()  # 予測値をソフトマックスで確率に変換
    loss = log_loss(y_true=labels, y_pred=probs)  # ロスの計算
    acc = accuracy_score(y_true=labels, y_pred=preds.argmax(-1))  # 精度の計算
    return {"acc": acc, "log_loss": loss}  # 精度とロスを返す

### データ分割

ここでは、トレーニングと評価を`id % 5`に基づいて分割します。


In [ ]:
folds = [
    (
        [i for i in range(len(ds)) if i % config.n_splits != fold_idx],  # トレーニングインデックス
        [i for i in range(len(ds)) if i % config.n_splits == fold_idx]  # 評価インデックス
    ) 
    for fold_idx in range(config.n_splits)
]

In [ ]:
train_idx, eval_idx = folds[config.fold_idx]  # トレーニングと評価のインデックス取得

trainer = Trainer(
    args=training_args,  # トレーナー引数
    model=model,  # モデル
    tokenizer=tokenizer,  # トークナイザー
    train_dataset=ds.select(train_idx),  # トレーニングデータセット
    eval_dataset=ds.select(eval_idx),  # 評価データセット
    compute_metrics=compute_metrics,  # 指標計算関数
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),  # データのコラトレータ
)
trainer.train()  # トレーニングを開始

---

# コメント

> ## Yichuan Gao
> 
> このノートブックから結果を再現するのが難しいです。具体的には、このノートブックをダウンロードし、入力パスを変更しただけで、サンプル(100)を削除しました。
> 
> 4090でトレーニングした後、train_lossは約1.0までしか下がらず、eval_lossも約0.98で、主張されている0.937のCVロスよりもかなり劣っています。異なるマシンの異なるGPU構成や異なるバッチサイズでトレーニングを試みましたが、すべて似たような悪い結果でした。
> 
> 何が原因か非常に疑問に思っています。アドバイスがあれば歓迎します。
> 
> > ## skurita
> > 
> > 私も同様の問題を経験しています。
> > >
> > > ノートブックをダウンロードし、入力パスを変更し、サンプル(100)を削除しましたが、トレーニング/評価ロスは図のようにしか減少せず、主張されている0.937のCVロスよりも悪いです。
> > >
> > > "unsloth/gemma-2-9b-it-bnb-4bit"を4エポックでトレーニングし、各エポックの後にバリデーションを計算しました。
> > >
> > > このトレーニングノートブックから結果を再現する方法は誰か知っていますか？
> > >
> > > どんな洞察や提案も感謝されます。
> > >
> > > 著者[@emiz6413](https://www.kaggle.com/emiz6413)にも言及します。
> > >
> > > ありがとう。
> > >
> > > 

---

> ## S J Moudry
> > [@emiz6413](https://www.kaggle.com/emiz6413)  ノートブックの共有、ありがとうございます。非常に多くを学び、あなたの仕事に全て投票しました。いくつか質問があります：
> >
> > トレーニング後のモデルの評価精度はどのくらいでしたか？
> > トレーニングにはどのプラットフォームを使用していますか？ おすすめのものはありますか？
> > ハイパーパラメータの調整に関するヒントはありますか？ また、どのようにハイパーパラメータを決めましたか？ フルデータセットで何度もトレーニングしましたか、それとも効果を測るためにいくつかのテストランを行いましたか？
> >
> > > ## Eisuke MizutaniTopic Author
> > > 
> > > コメントありがとうございます。
> > > 
> > > - 1エポック後の評価セットのロスは0.9371でした。
> > > 
> > > - 私はpaperspaceを使用しています。合理的な固定価格で多くの実験を行いたい場合は、そのプラットフォームをお勧めします。
> > > 
> > > - ハイパーパラメータの調整にはあまり時間をかけておらず、gemma2のトレーニングには非常に長い時間がかかります。いくつかの実行を行い、学習曲線を見て、不可能そうなものは手動で中止しました。したがって、ハイパーパラメータを調整する余地はたくさんあると思います。
> > > 
> > > 

---

> ## Lorry Zou
> 
> なぜ以前公表したLlama3のトレーニングノートブックを再利用しなかったのですか？ モデル名/パスをllama3からgemma2に変更するだけでTPUで約6時間トレーニングできます。
> 
> > ## Eisuke MizutaniTopic Author
> > 
> > あなたが言っているのはkishanvavdaraのノートブックだと思います（私はトレーニングノートブックを発表していません）。
> > >
> > > 私はGPUでモデルをトレーニングしており、TPUで再現するかどうかは不明でした。 特に量子化はTPUではサポートされていません。 OOMが発生しない場合は量子化は必要ありませんが。
> > >
> > > あなたはTPUでgemma2をトレーニングすることができますか？
> > >
> > > > ## Lorry Zou
> > > > はい、私はKishanvavdaraのノートブックについて話していました。彼は以前に2つのGemma 2ノートブックを投稿しました[https://www.kaggle.com/code/kishanvavdara/gemma-2-9b-part-1](url)
> > > > アップロードされたデータセットとして"gemma-2-9b-hf"が含まれています。 TPUでトレーニングでき、早かったです。
> > > > 
> > > > > ## Eisuke MizutaniTopic Author
> > > > > > あなたはgemmaの最終隠れ状態の上にcatboost（または何かしらの）分類器をトレーニングしただけですか？
> > > > > > gemma2-9bの読み込みを試みるとOOMが発生し、量子化はTPUではサポートされていません。したがって、gemmaの重みを微調整するためにTPUの使用をあきらめざるを得ませんでした。
> > > > > > 

---

> ## Muhammad Haroon ul Hasnain
> 
> 素晴らしいノートブックと説明をありがとうございます。

---

> ## Mohamadreza Momeni
> 
> 素晴らしい仕事です。
> >
> > 親愛なる[@emiz6413](https://www.kaggle.com/emiz6413)への偉業です。

---

> ## floriandev
> 
> まず第一に[@emiz6413](https://www.kaggle.com/emiz6413)  このノートブックには大きな感謝を!!
> 
> 不幸にも、ファインチューニングの後、モデルをハギングフェイスにプッシュして、hfからのモデルを使用して推論ノートブックを呼び出す際に次のエラーが発生しました：
> 
> …このエラーが発生します…
> 
> > ## Lorry Zou
> > 
> > num_classes=3を指定しないと、デフォルト値は2になります。
> > 
> > > ## floriandev
> > > > こんにちはLorry、迅速な返信ありがとうございます!!
> > > > 
> > > > トレーニング/推論ノートブックは同じ著者のものです。変更を加えずにトレーニングに使用したため、精度やロスは問題なかったのですが、保存後に推論ノートブックに読み込むとエラーが出ます。
> > > > 
> > > > トレーニングされたモデルは、アーキテクチャ的に推論モデルと一致するはずです...うーん
> > > >
> > > > 推論ノートブックでクラス数をどのように導入するか、ということですが、新たにトレーニングされたモデルには1つの追加のパラメータがあるようです。
> > > > 
> > > > 教えてください、サポートの感謝します。
> > > > 
> > > > フロリアン。
> > > > 
> > > > > ## floriandev
> > > > > ああ、ローカルで生成されたチェックポイントを使用して（現在試しています）見つかりました...すぐに報告します！
> > > > 
> > > > > ## floriandev
> > > > > トレーニング出力がgemma_dirに使用されたため、問題が解決できます。
> > > > >
> > > > > 解決策：トレーニング出力をlora_dirとして使用しました。
> > > > >
> > > > 

---

> ## Nikhil Tumbde
> 
> ノートブックに感謝します。
> 
> fp16のトレーニング引数について質問がありますが、ファインチューニングの際にbf16を使用したらどうなりますか？ llama 3 8b（ベースモデル）で4ビットの量子化を使用したとき、数千ステップ後に勾配がNaNになりました。何か考えがありますか？

> > ## Eisuke MizutaniTopic Author
> > 
> > 私のデバイスで利用可能な場合は、bf16が一般的に安全なオプションだと思います。
> > 
> > 

---

> ## Vavilkin Alexander
> 
> こんにちは[@emiz6413](https://www.kaggle.com/emiz6413)! あなたのノートブックに感謝します、これはLLMのファインチューニングの実践に非常に役立ちます。モデル全体をどのように保存したか教えていただけますか？ "Trainer"で微調整する際、アダプターの重みしか保存されないと理解していますが、分類ヘッドの重みは保存されません。同時に推論ノートブックのモデルは通常の量子化モデルではなく、すでに分類用のヘッドが付いています。このことについてあなたからのコメントを非常に楽しみにしています。

> > ## Eisuke MizutaniTopic Author
> > 
> > 分類ヘッドはModulesToSaveWrapperとしてラップされているため、Trainerによって自動的に保存されます。
> > >
> > > チェックポイントからロードすると、トレーニングされた重みが分類ヘッドに読み込まれます。
> > >
> > > 

---

> ## Mattia Vanzetto
> 
> 質問してもいいですか？私はこれに対して初心者ですが、ノートブックが完了し出力が得られたら、出力を保存し、推論ノートブックで再読み込みする手順は何ですか？ ありがとうございます！

> > ## raconion
> > 
> > あなたのノートブックと同じレベルに出力フォルダがあるはずです。推論には最も大きな番号（ステップ）のサブフォルダを使用してください。この場合、5748であるべきです。
> > 
> > 

---

> ## raconion
> 
> こんにちは！ この素晴らしいノートブックをありがとう。私はH100*2でこのノートブックを実行した結果、あなたの結果では5748の代わりに11495ステップが出ました。また、cvスコアはやや悪化し0.964697です。
> 
> いくつかのコメントを読んだ後、バッチサイズに問題があると感じました。
> 
> 私の設定は次の通りです。
> 
> ```
> class Config:
>     output_dir: str = "output"  
>     checkpoint: str = "unsloth/gemma-2-9b-it-bnb-4bit"  # 4ビット量子化されたgemma-2-9b-instruct
>     max_length: int = 1024
>     n_splits: int = 5
>     fold_idx: int = 0
>     optim_type: str = "adamw_8bit"
>     per_device_train_batch_size: int = 2 
>     gradient_accumulation_steps: int = 2  # グローバルバッチサイズは8
>     per_device_eval_batch_size: int = 8
>     n_epochs: int = 1
>     freeze_layers: int = 16  # 総レイヤ数は42、最初の16層にはアダプターを追加しない
>     lr: float = 2e-4
>     warmup_steps: int = 20
>     lora_r: int = 16
>     lora_alpha: float = lora_r * 2 
>     lora_dropout: float = 0.05
>     lora_bias: str = "none" 
> ```
> 
> そして、
> 
> ```
> training_args = TrainingArguments(
>     output_dir="output",
>     overwrite_output_dir=True,
>     report_to="none",
>     num_train_epochs=config.n_epochs,
>     per_device_train_batch_size=config.per_device_train_batch_size,
>     gradient_accumulation_steps=config.gradient_accumulation_steps,
>     per_device_eval_batch_size=config.per_device_eval_batch_size,
>     logging_steps=10,
>     eval_strategy="epoch",
>     save_strategy="steps",
>     save_steps=200,
>     optim=config.optim_type,
>     fp16=True,
>     learning_rate=config.lr,
>     warmup_steps=config.warmup_steps,
> )
> ```
> 
> モデルを実際にトレーニングする際に異なる設定を使用していますか？ ありがとう！

> > ## Eisuke MizutaniTopic Author
> > 
> > あなたのコードで両方のGPUが表示されていますか？
> > > 
> > 

> > > > ## raconion
> > > > ノートブックのコードを直接再利用しています。デバイスマップによるとモデルの並列化が使用されているようです。nvidia-smiも両方のGPUが使用されていることを示しています。
> > > > 
> > > > デバイスマップ：
> > > > 
> > > > ```
> > > > {'model.embed_tokens': 0, 'model.layers.0': 0, 'model.layers.1': 0, 'model.layers.2': 0, 'model.layers.3': 0, 'model.layers.4': 0, 'model.layers.5': 0, 'model.layers.6': 0, 'model.layers.7': 0, 'model.layers.8': 0, 'model.layers.9': 0, 'model.layers.10': 0, 'model.layers.11': 0, 'model.layers.12': 0, 'model.layers.13': 0, 'model.layers.14': 0, 'model.layers.15': 0, 'model.layers.16': 1, 'model.layers.17': 1, 'model.layers.18': 1, 'model.layers.19': 1, 'model.layers.20': 1, 'model.layers.21': 1, 'model.layers.22': 1, 'model.layers.23': 1, 'model.layers.24': 1, 'model.layers.25': 1, 'model.layers.26': 1, 'model.layers.27': 1, 'model.layers.28': 1, 'model.layers.29': 1, 'model.layers.30': 1, 'model.layers.31': 1, 'model.layers.32': 1, 'model.layers.33': 1, 'model.layers.34': 1, 'model.layers.35': 1, 'model.layers.36': 1, 'model.layers.37': 1, 'model.layers.38': 1, 'model.layers.39': 1, 'model.layers.40': 1, 'model.layers.41': 1, 'model.norm': 1, 'score': 1}
> > > > ```
> > > > 
> > > > もしこの場合、以下のように
> > > > 
> > > > ```
> > > > per_device_train_batch_size: int = 2
> > > > gradient_accumulation_steps: int = 2
> > > > ```
> > > > グローバルバッチサイズは4になります。トレーニング時にグローバルバッチサイズが8になるようにper_device_train_batch_sizeとgradient_accumulation_stepsを異なる値に設定しますか？
> > > > 
> > > > > ## Eisuke MizutaniTopic Author
> > > > > > [@raconion](https://www.kaggle.com/raconion) 
> > > > > > 応答が遅れてすみません。
> > > > > > 
> > > > > > グローバルバッチサイズが8になるようにper_device_train_batch_sizeとgradient_accumulation_stepsを設定しています。グローバルバッチサイズが4だとスコアが悪化しました。
> > > > > > 
> > > 

---

> ## HZM
> 
> こんにちはEisuke、Llama3などの他のllmを試しましたか？ それは私にはうまくいきませんでした。
> 
> > ## Eisuke MizutaniTopic Author
> > 
> > Llama3を試しましたが、約0.98になりました。
> > 
> > 

---

> ## Kim Kumuha
> 
> トレーニングにはどれくらいの時間がかかりましたか？

---

> ## yuanzhe zhou
> 
> こんにちは、ノートブックの1エポック実行後の結果はどうなりますか？

> > ## Eisuke MizutaniTopic Author
> > 
> > 評価セット(id % 5 == 0)でのロスは0.9371です。
> > >
> > > チェックポイントを使用して結果を再現できます[こちら](https://www.kaggle.com/datasets/emiz6413/73zap2gx/data)。
> > >
> > > 
> > 
---